In [14]:
import matplotlib.pyplot as plt
import random as rd
from matplotlib.lines import Line2D
import seaborn as sns

In [2]:
%run Postproseccing.ipynb

Jobs Information:
Turbine ID      Crew Needed    
3               1              
4               3              
5               6              
6               7              
7               4              
8               9              
Vessel Types Information:
Vessel ID  Battery Range   Operational Cost     Speed (Knots)             Passenger Capacity   Fixed Cost          
1          2000            1                    20                        10                   5000                
2          3000            2                    22                        15                   10000               
3          4500            3.5                  25                        25                   17000               


In [11]:

def show_the_map(problem,title = None,centerradius=200, save_ = False, save_name = None):
    grid = problem["Locations"]
    n_clusters = len(problem["ChargingStations"])
    
    fig, ax = plt.subplots()
    # Extract x, y, and tags
    x_values = [point[0] for point in grid.values()][1+n_clusters:]
    y_values = [point[1] for point in grid.values()][1+n_clusters:]
    tags = list(grid.keys())[1+n_clusters:]

    # Plot the points and annotate with tags
    ax.scatter(x_values, y_values, marker='o', color='black',facecolor='none')
    #for tag, x, y in zip(tags, x_values, y_values):
    #    ax.text(x, y, str(tag), color='red', fontsize=9, ha='right', va='bottom')
        
    for radius in range(100, 1410 + 1, 100):
        circle = patches.Circle((0, 0), radius, edgecolor='black', fill=False, linestyle='dotted', alpha=0.5)
        ax.add_patch(circle)
        
    # Plot the centers and annotate with tags
    if n_clusters>0:
        cluster_x_values = [point[0] for point in grid.values()][1:1+n_clusters]
        cluster_y_values = [point[1] for point in grid.values()][1:1+n_clusters]
        ax.scatter(cluster_x_values, cluster_y_values, marker='X', color='orange')
        #for tag, x, y in zip(range(1,n_clusters+1), cluster_x_values, cluster_y_values):
        #    ax.text(x, y, f"CS{str(tag)}", color='black', fontsize=9, ha='right', va='bottom')
        
    ax.text(0, 0, 'D(0)', color='red', fontsize=10,fontweight='bold', ha='right', va='bottom')

    # Set labels and title
    
    if title:
        ax.set_title(f"Instance: {title}")
    else:
        ax.set_title('The problem map')
    
    ax.set_xlim(0, 1000)
    ax.set_ylim(0, 1000)
    
    circle = patches.Circle((0,0), centerradius, edgecolor = 'black',facecolor = 'lightgrey', alpha = 0.5)
    ax.add_patch(circle)
    
    if save_:
        plt.savefig(save_name)
        

    # Show the plot
    return ax
    

def plot_tour_on_map(ax, grid:dict,tour:list,color):
    tour_x = [grid[node][0] for node in tour]
    tour_y = [grid[node][1] for node in tour]
    

    ax.plot(tour_x, tour_y, linestyle='--', color=color, linewidth=1)
    
    
    
    return ax
    
def plot_all_trips(ax, solution, problem,t = None, save_ = False, save_name = None):
    
    locations = problem["Locations"]
    sol,v,_ = solution
    colors = ['green','red','blue']
    
    trips = split_solution(sol)
    
    trips = [x for x in trips if x]
    v = [x for x in v if x!=0]
    
    
    for i in range(len(colors)):
        ax.plot([], [], color=colors[i], label=f'Vessel {i+1}')
        
        
    for i in range(len(trips)):
        
        
        trip = trips[i]
        vessel_id = v[i]-1
        
        trip = [(0,0)] + trip + [(0,0)]
        
        tour = [x[0] for x in trip]
        
        
        plot_tour_on_map(ax,locations,tour,colors[vessel_id])
    
    ax.legend()
    
    if t:
        ax.set_title(t)
    else:
        ax.set_title('All trips')
    
    if save_:
        plt.savefig(save_name)
    
    
    return ax


def plot_some_trips(ax,solution,problem,n):
    locations = problem["Locations"]
    sol,v,_ = solution
    colors = ['green','red','blue']
    
    trips = split_solution(sol)
    
    trips = [x for x in trips if x]
    v = [x for x in v if x!=0]
    
    
    for i in range(len(colors)):
        ax.plot([], [], color=colors[i], label=f'Vessel {i+1}')
        
    
    p = rd.sample(range(len(trips)),min(n,len(trips)))
    
    for i in p:
        
        
        trip = trips[i]
        vessel_id = v[i]-1
        
        
        trip = [(0,0)] + trip + [(0,0)]
        
        tour = [x[0] for x in trip]
        
        print(tour)
        
        plot_tour_on_map(ax,locations,tour,colors[vessel_id])
    
    ax.legend()
    return ax
    
           
        


In [6]:
def show_vessel_distribution(solution):
    
    sol,vessels,_ = solution
    v = [x for x in vessels if x!=0]
    
    
    # List of vessel IDs
    vessel_ids = v

    # Count the frequency of each vessel ID
    vessel_counts = [vessel_ids.count(vessel_id) for vessel_id in vessel_ids]

    # Plot the vessel frequency
    plt.bar(vessel_ids, vessel_counts)
    plt.xlabel('Vessel ID')
    plt.ylabel('Frequency')
    plt.title('Vessel Usage Frequency')

    plt.show()
    

In [7]:
def plot_output_lists(costs,all_weights,all_scores):    
    # Plotting costs
    plt.figure(figsize=(10, 6))
    plt.plot(costs, linewidth=1)
    plt.xlabel('Iteration')
    plt.ylabel('Cost')
    plt.title('Evolution of Costs')
    plt.show()

    # Plotting weights
    plt.figure(figsize=(10, 6))
    for i in range(len(all_weights[0])):
        plt.plot([weights[i] for weights in all_weights], label=f'Operator {i+1}')
    plt.xlabel('Segment')
    plt.ylabel('Weight')
    plt.title('Evolution of Weights')
    plt.legend()
    plt.show()

    # Sum of all weights
    sum_weights = [sum(weights) for weights in all_weights]
    plt.figure(figsize=(10, 6))
    plt.plot(sum_weights)
    plt.xlabel('Segment')
    plt.ylabel('Sum of Weights')
    plt.title('Evolution of Sum of Weights')
    plt.show()

    # Plotting scores
    plt.figure(figsize=(10, 6))
    for i in range(len(all_scores[0])):
        plt.plot([scores[i] for scores in all_scores], label=f'Operator {i+1}')
    plt.xlabel('Segment')
    plt.ylabel('Score')
    plt.title('Evolution of Scores')
    plt.legend()
    plt.show()

In [1]:
def plot_costs(cost_movement, title='Cost over iterations', save=False, save_name='costs.png'):
    fig, ax = plt.subplots(figsize=(16, 8))#
    ax.plot(cost_movement)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Cost')
    ax.set_title(title)
    if save:
        plt.savefig(save_name)
    plt.show()
    
    return ax


In [3]:
def multiple_plots(plot1,plot2, title='Plots',save=False, save_name='multiple.png'):
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.plot(plot1, color='blue')
    ax.plot(plot2, color='red')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Cost')
    ax.set_title(title)
    if save:
        fig.savefig(save_name)
    plt.show()
    
    return ax

In [3]:
def zoomed_plot(data, x_min, x_max, save=False, save_name='zoomed_plot.png', data2=None):
    fig, ax = plt.subplots(figsize=(24, 14))
    
    ax.plot(data, color='blue')
    
    if data2:
        ax.plot(data2, color='red')
    
    ax.set_xlabel('Iteration', fontsize=22)
    ax.set_ylabel('Objective Value', fontsize=22)
    ax.set_ylim(min(data) * 0.9, max(data))
    
    
    
  
    # Create inset plot
    ax_inset = fig.add_axes([0.35, 0.35, 0.5, 0.5])
    ax_inset.plot(range(x_min, x_max), data[x_min:x_max], color='blue', linewidth=1)
    if data2:
        ax_inset.plot(range(x_min, x_max), data2[x_min:x_max], color='red', linewidth=1)
    
    ax_inset.set_ylim(min(data[x_min:x_max]), max(data[x_min:x_max]))
    ax_inset.set_xlim(x_min-10, x_max)
    
    
    #bbox = ax_inset.get_position()

    # Extract coordinates
    #scaled_bbox_x0 = bbox.x0 * len(data)
    #scaled_bbox_y0 = bbox.y0 * max(data)
    #scaled_bbox_x1 = bbox.x1 * len(data)
    #scaled_bbox_y1 = bbox.y1 * max(data)
    
    inset_min = min(data[x_min:x_max])
    inset_max = max(data[x_min:x_max])
    
    fig_x_min, fig_x_max = fig.gca().get_xlim()

    # Define scaling factors
    scaling_factor_x = len(data) / 100
    scaling_factor_y = (max(data)-min(data)) / 100

    # Line from x-axis at x_min to top-right corner of entire plot
    line1 = Line2D([0* scaling_factor_x, 27 * scaling_factor_x], [0 * scaling_factor_y+ min(data) * 0.9,min(data) * 0.9 +  33 * scaling_factor_y], linestyle='--', color='orange')

    # Line from x-axis at x_max to top-right corner of entire plot
    line2 = Line2D([10 * scaling_factor_x, 98 * scaling_factor_x], [min(data) * 0.9 + 0 * scaling_factor_y,min(data) * 0.9 +  33 * scaling_factor_y], linestyle='--', color='orange')

    # Line from y-axis at min(data[x_min:x_max]) to top-right corner of entire plot
    line3 = Line2D([fig_x_min-230, 27 * scaling_factor_x], [inset_min,min(data) * 0.9 +  33 * scaling_factor_y], linestyle='--', color='orange')

    # Line from y-axis at max(data[x_min:x_max]) to top-right corner of entire plot
    line4 = Line2D([fig_x_min-230, 27 * scaling_factor_x], [inset_max-5000,min(data) * 0.9 + 103 * scaling_factor_y], linestyle='--', color='orange')
            
    ax.add_line(line1)
    ax.add_line(line2)
    ax.add_line(line3)
    ax.add_line(line4)
    
    if data2:
        ax.set_title('Plot of the critical part of  the search. Baseline (blue) ExIII (red)', fontsize=22)
    else:
        ax.set_title('Plot of critical part of search', fontsize=22)
    
    if save:
        plt.savefig(save_name)
    
    plt.show()

In [9]:
def operators_scatter(deltas, operators, highlight=None, save=False, save_name='operators_scatter.png'):
    
    if not highlight:
        highlight = list(range(len(operators)))
    x = [item[0] for item in deltas if item[2] in highlight]
    y = [item[1] for item in deltas if item[2] in highlight]
    operator_idx = [item[2] for item in deltas if item[2] in highlight]
    
    plt.figure(figsize=(20, 10))
    
    scatter = plt.scatter(x, y, c=operator_idx, cmap='tab20', edgecolor='k')
    plt.colorbar(scatter, label='Operator Index')

    # Adding labels and title
    plt.xlabel('Iteration Number')
    plt.ylabel('Difference in Objective Value')
    plt.title('Scatter Plot to highlight operator performance')
    plt.grid(True)
    
    
    
    # Create legend
    legend_labels = [operators[idx].__name__ for idx in range(len(operators)) if idx in highlight]
    print(legend_labels)
    plt.legend(legend_labels, loc='lower right')
    
    plt.show()
    
    if save:
        plt.savefig(save_name)
    
    

In [2]:
def exII_results_plot(exII, save_ = False, save_name = '', initial_cost = None):
    _, best_costs,avg_costs, n_charges_lists, amnt_charged_lists, vessel_count_lists, n_vessels = exII

    # Plot 1: Best Cost
    fig1, ax1 = plt.subplots(figsize=(10, 4))
    ax1.plot(n_vessels, best_costs, marker='o', color='blue')
    ax1.set_title('Best cost for different vessel counts')
    ax1.set_xticks(n_vessels)
    ax1.set_xlabel('Number of vessels')
    ax1.set_ylabel('Objective value')
    if initial_cost:
        ax1.axhline(y=initial_cost, color='black', linestyle='--', label='Initial cost')
        #ax1.legend(loc='upper left')
    if save_:
        fig1.savefig(save_name + '_best_costs_plot.png')
        
            # Plot 1: Best Cost
    fig2, ax2 = plt.subplots(figsize=(10, 4))
    ax2.plot(n_vessels, avg_costs, marker='o', color='blue')
    ax2.set_title('Average costs for different vessel counts')
    ax2.set_xticks(n_vessels)
    ax2.set_xlabel('Number of vessels')
    ax2.set_ylabel('Objective value')
    if initial_cost:
        ax2.axhline(y=initial_cost, color='black', linestyle='--', label='Initial cost')
        #ax2.legend(loc='upper left')
    if save_:
        fig2.savefig(save_name + '_average_costs_plot.png')
    

    # Plot 2: Box Plot with Overlay Line Plot of Charges
    df, avgs = data_prep(n_vessels, n_charges_lists)
    fig2, ax2 = plt.subplots(figsize=(10, 4))
    sns.boxplot(x='X-Value', y='Value', data=df, color='lightgray', ax=ax2)
    categories = df['X-Value'].astype(str).unique()
    category_positions = range(len(categories))
    ax2.plot(category_positions, avgs, 'o-', color='red', label='Average Value')
    ax2.set_xticks(category_positions)
    ax2.set_xticklabels(categories)
    ax2.set_xlabel('Number of Vessels')
    ax2.set_ylabel('Frequency of Charging')
    
    ax2.legend(loc='upper left')
    if save_:
        fig2.savefig(save_name + '_charging_frequency_boxplot.png')
    

    # Plot 3: Box Plot with Overlay Line Plot of Amount Charged
    df, avgs = data_prep(n_vessels, amnt_charged_lists)
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    sns.boxplot(x='X-Value', y='Value', data=df, color='lightgray', ax=ax3)
    ax3.plot(category_positions, avgs, 'o-', color='red', label='Average Value')
    ax3.set_xticks(category_positions)
    ax3.set_xticklabels(categories)
    ax3.set_xlabel('Number of Vessels')
    ax3.set_ylabel('Distance Charged')
    
    ax3.legend(loc='upper left')
    if save_:
        fig3.savefig(save_name + '_amount_charged_boxplot.png')
    

    # Plot 4: Stacked Bar Plot
    vessel_count_array = np.array(vessel_count_lists)
    average_list = np.mean(vessel_count_array, axis=1)
    bar_data = {'Parameter': n_vessels, 'Values': average_list}
    colors = ['green', 'red', 'blue']
    fig4, ax4 = plt.subplots(figsize=(10, 4))
    for i, param in enumerate(bar_data['Parameter']):
        bottoms = np.cumsum(bar_data['Values'][i]) - bar_data['Values'][i]
        for j in range(len(bar_data['Values'][i])):
            ax4.bar(param, bar_data['Values'][i][j], bottom=bottoms[j], color=colors[j], width=0.5, label=f'Vessel {j+1}' if i == 0 else "")
    ax4.plot(n_vessels,n_vessels, color = 'black', linestyle = '--',label = 'Maximum number of vessels')
    ax4.set_xlabel('Number of Vessels')
    ax4.set_ylabel('Vessel usage frequency')
    ax4.set_xticks(n_vessels)
    ax4.legend()
    if save_:
        fig4.savefig(save_name + '_vessel_usage_bar_plot.png')
        
        # New Plot: Box Plot with Overlay Line Plot of Amount Charged per Visit
    amnt_per_charge_lists = [[amnt / n if n > 0 else 0 for amnt, n in zip(amnt_charged, n_charges)]
                             for amnt_charged, n_charges in zip(amnt_charged_lists, n_charges_lists)]
    
    df, avgs = data_prep(n_vessels, amnt_per_charge_lists)
    fig5, ax5 = plt.subplots(figsize=(10, 4))
    sns.boxplot(x='X-Value', y='Value', data=df, color='lightgray', ax=ax5)
    ax5.plot(category_positions, avgs, 'o-', color='red', label='Average Value')
    ax5.set_xticks(category_positions)
    ax5.set_xticklabels(categories)
    ax5.set_xlabel('Number of Vessels')
    ax5.set_ylabel('Amount Charged per Visit')
    ax5.legend(loc='upper left')
    if save_:
        fig5.savefig(save_name + '_amount_per_charge_boxplot.png')


    plt.show()

In [2]:


def exII_results_comparison_plot(exII_1, exII_2, save_=False, save_name='', initial_cost=None):
    labels = ['Base specs', 'ExIII']
    results = [exII_1, exII_2]
    colors = ['blue', 'green']
    
    # Plot 1: Best Cost
    fig1, ax1 = plt.subplots(figsize=(10, 4))
    for i, result in enumerate(results):
        _, best_costs, avg_costs, n_charges_lists, amnt_charged_lists, vessel_count_lists, n_vessels = result
        ax1.plot(n_vessels, best_costs, marker='o', color=colors[i], label=f'{labels[i]}')
    ax1.set_title('Best cost for different vessel counts')
    ax1.set_xticks(n_vessels)
    ax1.set_xlabel('Number of vessels')
    ax1.set_ylabel('Objective value')
    if initial_cost is not None:
        ax1.axhline(y=initial_cost, color='black', linestyle='--', label='Initial cost')
    ax1.legend(loc='upper left')
    
    # Plot 2: Average Cost
    fig2, ax2 = plt.subplots(figsize=(10, 4))
    for i, result in enumerate(results):
        _, best_costs, avg_costs, n_charges_lists, amnt_charged_lists, vessel_count_lists, n_vessels = result
        ax2.plot(n_vessels, avg_costs, marker='o', color=colors[i], label=f'{labels[i]}')
    ax2.set_title('Average costs for different vessel counts')
    ax2.set_xticks(n_vessels)
    ax2.set_xlabel('Number of vessels')
    ax2.set_ylabel('Objective value')
    if initial_cost is not None:
        ax2.axhline(y=initial_cost, color='black', linestyle='--', label='Initial cost')
    ax2.legend(loc='upper left')
    

    # Plot 3: Average Frequency of Charging
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    for i, result in enumerate(results):
        _, best_costs, avg_costs, n_charges_lists, amnt_charged_lists, vessel_count_lists, n_vessels = result
        df, avgs = data_prep(n_vessels, n_charges_lists)
        categories = df['X-Value'].astype(str).unique()
        category_positions = range(len(categories))
        ax3.plot(category_positions, avgs, 'o-', color=colors[i], label=f'{labels[i]}')
    ax3.set_title('Average Frequency of Charging for Different Vessel Counts')
    ax3.set_xticks(category_positions)
    ax3.set_xticklabels(categories)
    ax3.set_xlabel('Number of Vessels')
    ax3.set_ylabel('Frequency of Charging')
    ax3.legend(loc='upper left')
   

    # Plot 4: Average Amount Charged
    fig4, ax4 = plt.subplots(figsize=(10, 4))
    for i, result in enumerate(results):
        _, best_costs, avg_costs, n_charges_lists, amnt_charged_lists, vessel_count_lists, n_vessels = result
        df, avgs = data_prep(n_vessels, amnt_charged_lists)
        categories = df['X-Value'].astype(str).unique()
        category_positions = range(len(categories))
        ax4.plot(category_positions, avgs, 'o-', color=colors[i], label=f'{labels[i]}')
    ax4.set_title('Average Amount Charged for Different Vessel Counts')
    ax4.set_xticks(category_positions)
    ax4.set_xticklabels(categories)
    ax4.set_xlabel('Number of Vessels')
    ax4.set_ylabel('Distance Charged')
    ax4.legend(loc='upper left')
   

    # Plot 5: Stacked Bar Plot of Vessel Usage
    fig5, ax5 = plt.subplots(figsize=(10, 4))
    bar_width = 0.4
    bar_positions_1 = np.array(n_vessels) - (bar_width/2)
    bar_positions_2 = np.array(n_vessels) + (bar_width/2)
    colors = ['green', 'red', 'blue', 'purple', 'orange']
    for i, result in enumerate(results):
        _, best_costs, avg_costs, n_charges_lists, amnt_charged_lists, vessel_count_lists, n_vessels = result
        vessel_count_array = np.array(vessel_count_lists)
        average_list = np.mean(vessel_count_array, axis=1)
        bar_positions = bar_positions_1 if i == 0 else bar_positions_2
        for j in range(average_list.shape[1]):
            bottoms = np.sum(average_list[:, :j], axis=1)
            ax5.bar(bar_positions, average_list[:, j], bar_width, bottom=bottoms, color=colors[j % len(colors)],edgecolor='white', label=f'Vessel {j+1}' if i == 0 else "")
    ax5.plot(n_vessels, n_vessels, color='black', linestyle='--', label='Maximum number of vessels')
    ax5.set_xlabel('Number of Vessels')
    ax5.set_ylabel('Vessel usage frequency')
    ax5.set_title('Base specs (left)      ExIII (right)')
    ax5.set_xticks(n_vessels)
    ax5.legend()
    
    if save_:
        fig1.savefig(f'{save_name}_best_costs_comparison_plot.eps', format='eps')
        fig2.savefig(f'{save_name}_average_costs_comparison_plot.eps', format='eps')
        fig3.savefig(f'{save_name}_charging_frequency_comparison_plot.eps', format='eps')
        fig4.savefig(f'{save_name}_amount_charged_comparison_plot.eps', format='eps')
        fig5.savefig(f'{save_name}_vessel_usage_comparison_bar_plot.eps', format='eps')


    plt.show()


In [1]:
#HELPER:
    
def data_prep(x,y):
    
    # Create a DataFrame
    df = pd.DataFrame(y, index=x).T  # Transpose to make each row an observation

    # Melt the DataFrame for easier plotting, using x_values for group names
    df_melted = df.melt(var_name='X-Value', value_name='Value')

    # Calculate averages for the line plot
    averages = df.mean().values
    
    return df_melted,averages